In [2]:
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("[INST] What is the meaning of {word}?[/INST]")

llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={
        "max_new_tokens": 250,
    },
)


chain = prompt | llm


chain.invoke({"word": "potato"})

' Potato is a starchy vegetable that is grown underground in rows, in well-drained soil, usually in clusters called "tubers." It is a popular food source worldwide, and is used in a variety of dishes, including soups, salads, stews, and as a side dish. Potatoes are also used for industrial purposes, such as the production of starch and biofuels. The word "potato" comes from the Spanish word "patata," which means "potato" in English.'